<a href="https://colab.research.google.com/github/arielwendichansky/Copa-America-Model-2024/blob/main/Copa_America_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CopaAnalyzer: Unveiling the next Copa America Champion with ML**

In the heart of every passionate football enthusiast lies an insatiable desire to witness the beautiful game's unpredictable drama unfold on the grandest stages. As an Argentine, my love for football runs deep, particularly when the pride of my nation takes center stage, led by the GOAT Lionel Messi. The 2022 World Cup victory only fueled my passion and sparked a new endeavor—to create a predictive model that forecasts the next Copa America champion.

CopaAnalyzer utilizes logistic regression to peer into the forthcoming tournament's outcome. Drawing from three primary data sources (historical games data since 2020, FIFA rankings, and team skill) this model aims to unravel the intricacies of South American football dynamics.

This project serves as the culmination of my journey through a data analytics boot camp, showcasing the application of tools and techniques acquired along the way. From data wrangling to model evaluation, each step reflects a commitment to harnessing the power of data to unlock footballing insights.

Join me as we dive into the realm of CopaAnalyzer, where analytics meets anticipation, and together, let's celebrate the enduring magic of football.

Let the ball start rolling!

# Installs

In [56]:
# Libraries to scrap info from the web
! pip install selenium
! pip install requests beautifulsoup4 pandas
# Library for ML
! pip install scikit-learn

In [57]:
# Library to scrap info from different websites related to football.
! pip install LanusStats
! pip install --upgrade LanusStats

In [58]:
# Gemini model
!pip install -q -U google-generativeai

# Libraries

In [160]:
# To know todays date
from datetime import date

# To handle the data
import pandas as pd
import numpy as np
import random
import joblib

# For web scraping
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
import requests as re
import LanusStats as ls

# To visualize the data
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# To preprocess the data and divide the data
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score,recall_score, classification_report, confusion_matrix, roc_auc_score

# Machine learning model
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Null values
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer # For categorical values

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Gemini packages
import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro')

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Web Scraping

Disclaimer: The website from where the data is being scraped is continuously updating so this might change the data extracted

In [5]:
# URL of the webpage to scrape
url = 'https://www.ole.com.ar/copa-america/copa-america-2024-listas-convocados-argentina-messi_0_7KCGeSguIO.html'

# Send a GET request to the URL
response = re.get(url)

# Parse the HTML content of the webpage
soup = bs(response.text, 'html.parser')

# Find all div elements with class "custom-text"
news_items = soup.find_all("div", class_="custom-text")

# Create a dictionary to store the titles and content for each country
country_news = {}

# Initialize variables to store the current country and its player list
current_country = None
player_list = ""
skip_paragraph = False  # Flag to determine if the paragraph should be skipped

# Iterate through each news item
for item in news_items:
    # Find the h2 element to extract the title (country)
    title_element = item.find("h2")
    if title_element:
        # If there's a title, update the current country and reset the player list
        if current_country:
            country_news[current_country] = player_list.strip()
            player_list = ""
        current_country = title_element.text.strip()
        skip_paragraph = True  # Skip the first paragraph after the title
    else:
        # If there's no title, check if the paragraph should be skipped
        if skip_paragraph:
            skip_paragraph = False
            continue

        # Add the content to the player list
        content_paragraphs = item.find_all("p")
        player_list += "\n".join([p.text.strip() for p in content_paragraphs]) + "\n"

# Add the last country and its player list to the dictionary
if current_country:
    country_news[current_country] = player_list.strip()

# Print the dictionary with desired format
for country, players in country_news.items():
    print(country)
    print(players)
    print("----------------------")


GRUPO A
Se acerca la Copa América 2024 y las distintas selecciones definen sus planteles. Hasta acá, sólo Brasil y Ecuador anunciaron sus planteles completos con 26 jugadores para el torneo que se va a jugar en Estados Unidos. Antes y después, hubo otros seleccionados que brindaron nóminas preliminares, con jugadores por cortar como es el caso de Argentina. La fecha límite para presentar las listas es el 15 de junio.
----------------------
Argentina
Arqueros: Emiliano Martínez (Aston Villa); Franco Armani (River Plate) y Gerónimo Rulli (Ajax).
Defensores: Gonzalo Montiel (Nottingham Forest); Nahuel Molina (Atlético Madrid); Leonardo Balerdi (Olympique de Marsella); Cristian Romero (Tottenham); Germán Pezzella (Real Betis); Lucas Martínez Quarta (Fiorentina); Nicolás Otamendi (Benfica); Lisandro Martínez (Manchester United); Marcos Acuña (Sevilla); Nicolás Tagliafico (Lyon) y Valentín Barco (Brighton).
Volantes: Guido Rodríguez (Real Betis); Leandro Paredes (Roma); Alexis Mac Allister (

In [6]:
# Define a function to extract player names and teams from the player list
def extract_players_and_teams(player_list, country):
    players = []
    lines = player_list.split('\n')
    category = ""
    for line in lines:
        if line.strip() == "":
            continue
        if ":" in line:
            category = line.split(":")[0].strip()
            players_name = line.split(":")[1].strip()
            # Replace ' y ' with ';' and ',' with ';' to standardize delimiters
            players_name = players_name.replace(' y ', ';').replace(',', ';')
            player_list = players_name.split(';')
            # Clean up any leading/trailing whitespace
            for player in player_list:
                players.append((country, player.strip(), category))
    return players

player_data = []

# Iterate through each country and its player list
for country, players_list in country_news.items():
    if country.startswith('GRUPO'):
        continue
    if players_list:  # Ensure there's player data to process
        country_players = extract_players_and_teams(players_list, country)
        player_data.extend(country_players)
        print(f"{len(country_players)} players added for country: {country}")

# Create a DataFrame from the player data list
df = pd.DataFrame(player_data, columns=['Country', 'Name (Team)', 'Category'])
print(df)


29 players added for country: Argentina
16 players added for country: Perú
54 players added for country: Chile
27 players added for country: Canadá
29 players added for country: México
26 players added for country: Ecuador
47 players added for country: Venezuela
26 players added for country: Jamaica
27 players added for country: Estados Unidos
21 players added for country: Uruguay
27 players added for country: Panamá
28 players added for country: Bolivia
26 players added for country: Brasil
28 players added for country: Colombia
27 players added for country: Paraguay
27 players added for country: Costa Rica
        Country                          Name (Team)    Category
0     Argentina      Emiliano Martínez (Aston Villa)    Arqueros
1     Argentina          Franco Armani (River Plate)    Arqueros
2     Argentina               Gerónimo Rulli (Ajax).    Arqueros
3     Argentina  Gonzalo Montiel (Nottingham Forest)  Defensores
4     Argentina      Nahuel Molina (Atlético Madrid)  Defens

From the website where the list of player where extracted some players does not have a ',' to divide each. Therefore, cleaning the data to have each row with a unique player is necessary.

In [14]:
# Function to split rows with multiple players and extract team names
def split_players(row):
    players = row['Name (Team)'].split(') ')
    new_rows = []
    for player in players:
        if player:
            if not player.endswith(')'):
                player
            parts = player.split(' (')
            if len(parts) == 2:
                name, team = parts
                new_rows.append([row['Country'], name.strip(), team.strip(), row['Category']])
    return new_rows

# Apply the function and create a new DataFrame
new_rows = []
for _, row in df.iterrows():
    new_rows.extend(split_players(row))

# Convert new rows into a DataFrame
new_df = pd.DataFrame(new_rows, columns=['Country', 'Name','Team', 'Position'])

# Remove ')' and '.' from 'Team' column
new_df['Team'] = new_df['Team'].str.replace(r')', '')
new_df['Team'] = new_df['Team'].str.replace(r'.', '')

# Replacing position names
new_df['Position'] = new_df['Position'].replace({'Arqueros': 'GK', 'Defensores': 'CB', 'Volantes': 'CM', 'Delanteros': 'CF'})

new_df.head()

,Country,Name,Team,Position
0,Argentina,Emiliano Martínez,Aston Villa,GK
1,Argentina,Franco Armani,River Plate,GK
2,Argentina,Gerónimo Rulli,Ajax,GK
3,Argentina,Gonzalo Montiel,Nottingham Forest,CB
4,Argentina,Nahuel Molina,Atlético Madrid,CB


## Scraping data for teams without official list

In [15]:
for country in country_news:
  if country.startswith('GRUPO'):
    continue
  else:
    if country not in new_df['Country'].unique():

      print(country)

(Date 2024-06-05) These 4 countries have not presented the players who will play in the Copa America Edition 2024 yet (I will take the shortlist from their last international match).

(Date 2024-06-07) As mentioned, the page from where the data is being scraped is frequenty updated with new information. Therefore, the 4 countries in which the list of players was missing (Canada, Jamaica, Panama, and Uruguay) are already up-to-date. It will be possible to see what was my previous idea, but I will keep it with the official list of players.

### Uruguay

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

# URL of the webpage to scrape
url = 'https://www.vozdeamerica.com/a/uruguay-confirma-convocados-amistoso-mexico-copa-america/7640452.html'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful.")
else:
    print("Failed to retrieve page:", response.status_code)
    exit()

# Parse the HTML content of the webpage
soup = bs(response.text, 'html.parser')

# Create a dictionary to store the positions and players for Uruguay
uruguay_squad = {}

# Find the section containing player positions
content_section = soup.find('div', id='article-content')

# List of valid positions to ensure only these are included
valid_positions = ['Porteros:', 'Defensas:', 'Mediocampistas:', 'Delanteros:']

if content_section:
    # Get all paragraphs in the content section
    paragraphs = content_section.find_all('p')

    current_position = None
    for paragraph in paragraphs:
        # Check if the paragraph contains a position
        position_tag = paragraph.find('strong')
        if position_tag:
            position_text = position_tag.text.strip()
            if position_text in valid_positions:
                current_position = position_text
                uruguay_squad[current_position] = []
        elif current_position:
            player_text = paragraph.text.strip()
            if player_text:  # Avoid adding empty strings
                uruguay_squad[current_position].append(player_text)

# Adding players missing in the list online
uruguay_squad['Mediocampistas:'].append('- Federico Valverde (Real Madrid)')
uruguay_squad['Defensas:'].append('- Ronald Araújo (Barcelona)')

# Prepare data for DataFrame
data = []
for position, players in uruguay_squad.items():
    for player in players:
        name_team = player.split(' (')
        name = name_team[0].strip('-').strip()
        team = name_team[1].strip(')')
        data.append(['Uruguay', name, team, position.strip(':')])

# Creating a dataframe for the Uruguay squad
df_uruguay = pd.DataFrame(data, columns=['Country', 'Name', 'Team', 'Category'])

# Display the DataFrame
print(df_uruguay)

Request successful.
    Country                Name                 Team        Category
0   Uruguay       Sergio Rochet        Internacional        Porteros
1   Uruguay       Santiago Mele               Junior        Porteros
2   Uruguay   Sebastián Cáceres              América        Defensas
3   Uruguay  José María Giménez   Atlético de Madrid        Defensas
4   Uruguay         Lucas Olaza            Krasnodar        Defensas
5   Uruguay     Mathías Olivera               Napoli        Defensas
6   Uruguay      Nahitan Nández           Al-Qadsiah        Defensas
7   Uruguay       Ronald Araújo            Barcelona        Defensas
8   Uruguay       Manuel Ugarte  París Saint-Germain  Mediocampistas
9   Uruguay        César Araújo         Orlando City  Mediocampistas
10  Uruguay   Rodrigo Bentancur            Tottenham  Mediocampistas
11  Uruguay   Federico Valverde          Real Madrid  Mediocampistas
12  Uruguay   Facundo Pellistri              Granada      Delanteros
13  Uruguay  M

### Panama

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

# URL of the webpage to scrape
url = 'https://www.tudn.com/futbol/copa-america-2024/copa-america-2024-seleccion-panama-convocatoria-para-amistosos-espana'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful.")
else:
    print("Failed to retrieve page:", response.status_code)
    exit()

# Parse the HTML content of the webpage
soup = bs(response.text, 'html.parser')

# Create a dictionary to store the positions and players for Panama
panama_squad = {}

# Find the section containing player positions
content_sections = soup.find_all('div', class_='content-base articleBody col-span-full mb-8 tracking-[0.005em] sm:col-start-3 md:col-span-7 md:col-start-5 lg:col-span-6 lg:col-start-5')

# List of valid positions to ensure only these are included
valid_positions = ['Porteros', 'Defensas', 'Mediocampistas', 'Delanteros']


for section in content_sections:
    # Get all lists in the content section
    lists = section.find_all('li')
    current_position = None
    for li in lists:
        text = li.text.strip()
        if ':' in text:
            position = text.split(':')[0]
            if position in valid_positions:
                current_position = position
                panama_squad[current_position] = []
            players = text.split(':')[1].split('), ')
            for player in players:
                if current_position:
                    panama_squad[current_position].append(player)


# Prepare data for DataFrame
data = []
for position, players in panama_squad.items():
    for player in players:

        name_team = player.split(' (')
        name = name_team[0].strip()
        team = name_team[1].strip(')')
        data.append(['Panama', name, team, position.strip(':')])

# Creating a dataframe for the Panama squad
df_panama = pd.DataFrame(data, columns=['Country', 'Name', 'Team', 'Category'])

# Clean and display the DataFrame

df_panama['Team'] = df_panama['Team'].str.replace(r')', '')
df_panama['Team'] = df_panama['Team'].str.replace(r'(', '')
df_panama['Team'] = df_panama['Team'].str.replace(r'.', '')
df_panama['Team'] = df_panama['Team'].str.replace(r',', '')

# Function to keep only the first two words
def keep_first_two_words(team):
    words = team.split()
    if len(words) > 2:
        return ' '.join(words[:-1])
    else:
        return team


# Apply the function to the 'Team' column
df_panama['Team'] = df_panama['Team'].apply(keep_first_two_words)
print(df_panama)

Request successful.
   Country                 Name                  Team        Category
0   Panama     Orlando Mosquera      Maccabi Tel-Aviv        Porteros
1   Panama        Eddie Roberts      CA Independiente        Porteros
2   Panama       Andrés Andrade             LASK Linz        Defensas
3   Panama           Eric Davis             FC Kosice        Defensas
4   Panama      Michael Murillo    Olympique Marsella        Defensas
5   Panama       César Blackman     Slovan Bratislava        Defensas
6   Panama       Edgardo Fariña         Municipal GUA        Defensas
7   Panama      Roderick Miller        Turan Tovuz IK        Defensas
8   Panama          Orman Davis      CA Independiente        Defensas
9   Panama       Sergio Ramírez      CA Independiente        Defensas
10  Panama        Gabriel Brown             Dep Árabe        Defensas
11  Panama  José Luis Rodríguez          FC Famalicão  Mediocampistas
12  Panama       Édgar Bárcenas        Mazatlán FCMEX  Mediocampistas


### Canada

In order to use different tools, as well for facility, I will use Gemini AI to create the following lists

In [ ]:
response = model.generate_content(f''' For the following list of players from Canada create a dictionary (canada_players) with the following keys:
Country, Name, Team, Category.
ARQUEROS (4)
Maxime Crépeau - Portland Timbers
Thomas McGill - Brighton & Hove Albion FC
Dayne St. Clair - Minnesota United FC
Grégoire Swiderski - Girondins de Bordeaux B^
DEFENSORES (9)
Moïse Bombito - Colorado Rapids
Derek Cornelius - Malmö FF
Alphonso Davies - Bayern Munich
Luc de Fougerolles - Fulham FC
Kyle Hiebert - St. Louis CITY SC
Alistair Johnston - Celtic FC
Richie Laryea - Toronto FC
Kamal Miller - Portland Timbers
Dominick Zator - Korona Kielce
MEDIOCAMPISTAS (5)
Mathieu Choinière - CF Montréal
Stephen Eustáquio - FC Porto
Ismaël Koné - Watford FC
Jonathan Osorio - Toronto FC
Samuel Piette - CF Montréal
DELANTEROS (9)
Thelonius Bair - Motherwell
Charles-Andreas Brym - Sparta Rotterdam
Tajon Buchanan - Inter Milan
Jonathan David - LOSC Lille
Junior Hoilett - Aberdeen FC
Cyle Larin - RCD Mallorca
Liam Millar - FC Basel
Jacob Shaffelburg - Nashville SC
Iké Ugbo - ESTAC Troyes
''')
to_markdown(response.text)

> ```python
> canada_players = {
>     "Country": ["Canada", "Canada", "Canada", "Canada", "Canada", "Canada", "Canada", "Canada", "Canada", "Canada", "Canada", "Canada", "Canada", "Canada", "Canada", "Canada", "Canada", "Canada", "Canada", "Canada"],
>     "Name": ["Maxime Crépeau", "Thomas McGill", "Dayne St. Clair", "Grégoire Swiderski", "Moïse Bombito", "Derek Cornelius", "Alphonso Davies", "Luc de Fougerolles", "Kyle Hiebert", "Alistair Johnston", "Richie Laryea", "Kamal Miller", "Dominick Zator", "Mathieu Choinière", "Stephen Eustáquio", "Ismaël Koné", "Jonathan Osorio", "Samuel Piette", "Thelonius Bair", "Charles-Andreas Brym"],
>     "Team": ["Portland Timbers", "Brighton & Hove Albion FC", "Minnesota United FC", "Girondins de Bordeaux B", "Colorado Rapids", "Malmö FF", "Bayern Munich", "Fulham FC", "St. Louis CITY SC", "Celtic FC", "Toronto FC", "Portland Timbers", "Korona Kielce", "CF Montréal", "FC Porto", "Watford FC", "Toronto FC", "CF Montréal", "Motherwell", "Sparta Rotterdam"],
>     "Category": ["ARQUEROS", "ARQUEROS", "ARQUEROS", "ARQUEROS", "DEFENSORES", "DEFENSORES", "DEFENSORES", "DEFENSORES", "DEFENSORES", "DEFENSORES", "DEFENSORES", "DEFENSORES", "DEFENSORES", "MEDIOCAMPISTAS", "MEDIOCAMPISTAS", "MEDIOCAMPISTAS", "MEDIOCAMPISTAS", "MEDIOCAMPISTAS", "DELANTEROS", "DELANTEROS"]
> }
> ```

In [ ]:
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "```python\ncanada_players = {\n    \"Country\": [\"Canada\", \"Canada\", \"Canada\", \"Canada\", \"Canada\", \"Canada\", \"Canada\", \"Canada\", \"Canada\", \"Canada\", \"Canada\", \"Canada\", \"Canada\", \"Canada\", \"Canada\", \"Canada\", \"Canada\", \"Canada\", \"Canada\", \"Canada\"],\n    \"Name\": [\"Maxime Cr\u00e9peau\", \"Thomas McGill\", \"Dayne St. Clair\", \"Gr\u00e9goire Swiderski\", \"Mo\u00efse Bombito\", \"Derek Cornelius\", \"Alphonso Davies\", \"Luc de Fougerolles\", \"Kyle Hiebert\", \"Alistair Johnston\", \"Richie Laryea\", \"Kamal Miller\", \"Dominick Zator\", \"Mathieu Choini\u00e8re\", \"Stephen Eust\u00e1quio\", \"Isma\u00ebl Kon\u00e9\", \"Jonathan Osorio\", \"Samuel Piette\", \"Thelonius Bair\", \"Charles-Andreas Brym\"],\n    \"Tea

In [ ]:
# Access the content attribute from the response
content = response.candidates[0].content.parts[0].text

# Find the starting index of 'player_list ='
start_index = content.find('canada_players = ')

# Find the ending index of ']\n```'
end_index = content.find('```', start_index)

# Extract the substring containing player_list
player_list_str = content[start_index:end_index].strip()

# Evaluate the string as Python code to get the player_list dictionary
exec(player_list_str)

# Convert the player_list dictionary to a DataFrame
df_canada = pd.DataFrame(canada_players)


# Display the DataFrame
print(df_canada)

   Country                  Name                       Team        Category
0   Canada        Maxime Crépeau           Portland Timbers        ARQUEROS
1   Canada         Thomas McGill  Brighton & Hove Albion FC        ARQUEROS
2   Canada       Dayne St. Clair        Minnesota United FC        ARQUEROS
3   Canada    Grégoire Swiderski    Girondins de Bordeaux B        ARQUEROS
4   Canada         Moïse Bombito            Colorado Rapids      DEFENSORES
5   Canada       Derek Cornelius                   Malmö FF      DEFENSORES
6   Canada       Alphonso Davies              Bayern Munich      DEFENSORES
7   Canada    Luc de Fougerolles                  Fulham FC      DEFENSORES
8   Canada          Kyle Hiebert          St. Louis CITY SC      DEFENSORES
9   Canada     Alistair Johnston                  Celtic FC      DEFENSORES
10  Canada         Richie Laryea                 Toronto FC      DEFENSORES
11  Canada          Kamal Miller           Portland Timbers      DEFENSORES
12  Canada  

### Jamaica

It is important to note that I am not providing the teams where each player is listed currently and asked the AI model to browse for this.

In [ ]:
jamaica_response = model.generate_content(f''' For the following list of players from Jamaica create a dictionary (jamaica_players) with the following keys:
Country, Name, Team, Category.
Search on the web for the teams where each player is listed.
Porteros: C. Boyce-Clarke(21 años),  S. Davis(23 años), J. Hibbert(19 años), J. Waite(25 años)
Defensas: J. Bell(26 años), D. Bernard(23 años), T. Gray(21 años), M. Hector(31 años), G. Irving(25 años), D. Lembikisa(20 años), D. Lowe(31 años), A. Reid(17 años), R. King (22 años), G. Leigh(29 años)
Mediocampistas: K. Anderson(19 años), D. Johnson(31 años), K. Lambert (27 años), K. Palmer (27 años), F. Reid(32 años)
Delanteros: M. Antonio(34 años), D. Beckford(26 años), D. Campbell(20 años), R. Cephas(24 años), B. De Cordova-Reid (31 años), K. Dixon(19 años), A. Marshsall(26 años), S. Nicholson(27 años)
''')
to_markdown(jamaica_response.text)




> ```python
> jamaica_players = {
>     "Country": "Jamaica",
>     "Players": [
>         {
>             "Name": "C. Boyce-Clarke",
>             "Team": None,
>             "Category": "Porteros"
>         },
>         {
>             "Name": "S. Davis",
>             "Team": None,
>             "Category": "Porteros"
>         },
>         {
>             "Name": "J. Hibbert",
>             "Team": None,
>             "Category": "Porteros"
>         },
>         {
>             "Name": "J. Waite",
>             "Team": "Harrogate Town",
>             "Category": "Porteros"
>         },
>         {
>             "Name": "J. Bell",
>             "Team": "Newport County",
>             "Category": "Defensas"
>         },
>         {
>             "Name": "D. Bernard",
>             "Team": "Oldham Athletic",
>             "Category": "Defensas"
>         },
>         {
>             "Name": "T. Gray",
>             "Team": "Leyton Orient",
>             "Category": "Defensas"
>         },
>         {
>             "Name": "M. Hector",
>             "Team": "Chelsea",
>             "Category": "Defensas"
>         },
>         {
>             "Name": "G. Irving",
>             "Team": "Swindon Town",
>             "Category": "Defensas"
>         },
>         {
>             "Name": "D. Lembikisa",
>             "Team": "Northampton Town",
>             "Category": "Defensas"
>         },
>         {
>             "Name": "D. Lowe",
>             "Team": "Nottingham Forest",
>             "Category": "Defensas"
>         },
>         {
>             "Name": "A. Reid",
>             "Team": "Fulham",
>             "Category": "Defensas"
>         },
>         {
>             "Name": "R. King",
>             "Team": "Bristol Rovers",
>             "Category": "Defensas"
>         },
>         {
>             "Name": "G. Leigh",
>             "Team": "Newport County",
>             "Category": "Defensas"
>         },
>         {
>             "Name": "K. Anderson",
>             "Team": "Wigan Athletic",
>             "Category": "Mediocampistas"
>         },
>         {
>             "Name": "D. Johnson",
>             "Team": "Bolton Wanderers",
>             "Category": "Mediocampistas"
>         },
>         {
>             "Name": "K. Lambert",
>             "Team": "Bristol City",
>             "Category": "Mediocampistas"
>         },
>         {
>             "Name": "K. Palmer",
>             "Team": "Portsmouth",
>             "Category": "Mediocampistas"
>         },
>         {
>             "Name": "F. Reid",
>             "Team": "Crystal Palace",
>             "Category": "Mediocampistas"
>         },
>         {
>             "Name": "M. Antonio",
>             "Team": "West Ham",
>             "Category": "Delanteros"
>         },
>         {
>             "Name": "D. Beckford",
>             "Team": "Newport County",
>             "Category": "Delanteros"
>         },
>         {
>             "Name": "D. Campbell",
>             "Team": "Luton Town",
>             "Category": "Delanteros"
>         },
>         {
>             "Name": "R. Cephas",
>             "Team": "Shrewsbury Town",
>             "Category": "Delanteros"
>         },
>         {
>             "Name": "B. De Cordova-Reid",
>             "Team": "Fulham",
>             "Category": "Delanteros"
>         },
>         {
>             "Name": "K. Dixon",
>             "Team": "Dundee United",
>             "Category": "Delanteros"
>         },
>         {
>             "Name": "A. Marshsall",
>             "Team": "Queen of the South",
>             "Category": "Delanteros"
>         },
>         {
>             "Name": "S. Nicholson",
>             "Team": "Bristol Rovers",
>             "Category": "Delanteros"
>         },
>     ]
> }
> ```

In [ ]:
jamaica_response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "```python\njamaica_players = {\n    \"Country\": \"Jamaica\",\n    \"Players\": [\n        {\n            \"Name\": \"C. Boyce-Clarke\",\n            \"Team\": None,\n            \"Category\": \"Porteros\"\n        },\n        {\n            \"Name\": \"S. Davis\",\n            \"Team\": None,\n            \"Category\": \"Porteros\"\n        },\n        {\n            \"Name\": \"J. Hibbert\",\n            \"Team\": None,\n            \"Category\": \"Porteros\"\n        },\n        {\n            \"Name\": \"J. Waite\",\n            \"Team\": \"Harrogate Town\",\n            \"Category\": \"Porteros\"\n        },\n        {\n            \"Name\": \"J. Bell\",\n            \"Team\": \"Newport County\",\n            \"Category\": \"Defensas\"\n        },\n    

In [ ]:
# Access the content attribute from the response
jamaica_content = jamaica_response.candidates[0].content.parts[0].text

# Find the starting index of 'player_dict ='
start_index = jamaica_content.find('jamaica_players = ')

# Find the ending index of ']\n```'
end_index = jamaica_content.find('```', start_index)

# Extract the substring containing player_list
player_list_str = jamaica_content[start_index:end_index].strip()

# Evaluate the string as Python code to get the player_list dictionary
exec(player_list_str)

# Convert the player_list dictionary to a DataFrame
df_jamaica = pd.DataFrame(jamaica_players['Players'])
df_jamaica['Country'] = 'Jamaica'
df_jamaica = df_jamaica[['Country','Name','Team','Category']]
# Display the DataFrame
print(df_jamaica)

    Country                Name                Team        Category
0   Jamaica     C. Boyce-Clarke                None        Porteros
1   Jamaica            S. Davis                None        Porteros
2   Jamaica          J. Hibbert                None        Porteros
3   Jamaica            J. Waite      Harrogate Town        Porteros
4   Jamaica             J. Bell      Newport County        Defensas
5   Jamaica          D. Bernard     Oldham Athletic        Defensas
6   Jamaica             T. Gray       Leyton Orient        Defensas
7   Jamaica           M. Hector             Chelsea        Defensas
8   Jamaica           G. Irving        Swindon Town        Defensas
9   Jamaica        D. Lembikisa    Northampton Town        Defensas
10  Jamaica             D. Lowe   Nottingham Forest        Defensas
11  Jamaica             A. Reid              Fulham        Defensas
12  Jamaica             R. King      Bristol Rovers        Defensas
13  Jamaica            G. Leigh      Newport Cou

## Data Merge

Merging all the df from the different countries into a single one (Date: 2024-06-05)

In [ ]:
# Create a list of DataFrames
dfs = [new_df, df_uruguay, df_panama, df_canada, df_jamaica]

# Concatenate the DataFrames along the rows
players_ca_24 = pd.concat(dfs, ignore_index=True)

# Display the merged DataFrame
print(players_ca_24)

       Country                Name               Team    Category
0    Argentina   Emiliano Martínez        Aston Villa    Arqueros
1    Argentina       Franco Armani        River Plate    Arqueros
2    Argentina      Gerónimo Rulli               Ajax    Arqueros
3    Argentina     Gonzalo Montiel  Nottingham Forest  Defensores
4    Argentina       Nahuel Molina    Atlético Madrid  Defensores
..         ...                 ...                ...         ...
537    Jamaica  Joel Latibeaudiere                     Delanteros
538    Jamaica        Kasey Palmer                     Delanteros
539    Jamaica      Karoy Anderson                     Delanteros
540    Jamaica      Devon Williams                     Delanteros
541    Jamaica       Kevon Lambert                     Delanteros

[542 rows x 4 columns]


In [ ]:
players_ca_24.to_csv('players_ca_24.csv') # Saving data for future analysis

(Date: 2024-06-07) The dataframe that will be used is the new_df as this one has the official list of players that will play the next Copa America championship.

# Info from Kaggle

In [ ]:
from google.colab import userdata
userdata.get('Kaggle')

In [ ]:
! kaggle datasets download -d nyagami/fc-24-players-database-and-stats-from-easports

In [ ]:
!unzip fc-24-players-database-and-stats-from-easports.zip

In [20]:
# Taking the Overall weight for each player from the game FC24 where they scores every player based on their skills
player_rating = pd.read_csv('male_players.csv')
player_rating.head()

,Unnamed: 0,Name,Nation,Club,Position,Age,Overall,Pace,Shooting,Passing,...,Strength,Aggression,Att work rate,Def work rate,Preferred foot,Weak foot,Skill moves,URL,Gender,GK
0,0,Kylian Mbappé,France,Paris SG,ST,24,91,97,90,80,...,77,64,High,Low,Right,4,5,https://www.ea.com/games/ea-sports-fc/ratings/...,M,NaN
1,1,Erling Haaland,Norway,Manchester City,ST,23,91,89,93,66,...,93,87,High,Medium,Left,3,3,https://www.ea.com/games/ea-sports-fc/ratings/...,M,NaN
2,2,Kevin De Bruyne,Belgium,Manchester City,CM,32,91,72,88,94,...,74,75,High,Medium,Right,5,4,https://www.ea.com/games/ea-sports-fc/ratings/...,M,NaN
3,3,Lionel Messi,Argentina,Inter Miami CF,CF,36,90,80,87,90,...,68,44,Low,Low,Left,4,4,https://www.ea.com/games/ea-sports-fc/ratings/...,M,NaN
4,4,Karim Benzema,France,Al Ittihad,CF,35,90,79,88,83,...,82,63,Medium,Medium,Right,4,4,https://www.ea.com/games/ea-sports-fc/ratings/...,M,NaN


In [ ]:
! kaggle datasets download -d hamzaadhnanshakir/international-football-tournament-results

In [ ]:
! unzip international-football-tournament-results.zip # Taking all the previous results for copa america so to have historical data

In [254]:
hs_df = pd.read_csv('copa_america.csv')
hs_df.head()

,Year,Date,Home Team,Away Team,Home Score,Away Score,Shootout,Tournament,City,Country,Neutral Venue,Winning Team,first_shooter,Losing Team
0,1916,1916-07-02,Chile,Uruguay,0.0,4.0,False,Copa América,Buenos Aires,Argentina,True,Uruguay,NaN,Chile
1,1916,1916-07-06,Argentina,Chile,6.0,1.0,False,Copa América,Buenos Aires,Argentina,False,Argentina,NaN,Chile
2,1916,1916-07-08,Brazil,Chile,1.0,1.0,False,Copa América,Buenos Aires,Argentina,True,Draw,NaN,Draw
3,1916,1916-07-10,Argentina,Brazil,1.0,1.0,False,Copa América,Buenos Aires,Argentina,False,Draw,NaN,Draw
4,1916,1916-07-12,Brazil,Uruguay,1.0,2.0,False,Copa América,Buenos Aires,Argentina,True,Uruguay,NaN,Brazil


In [ ]:
! kaggle datasets download -d cashncarry/fifaworldranking

In [ ]:
! unzip fifaworldranking.zip # Taking FIFA ranking for every team

In [278]:
rankining_FIFA = pd.read_csv('fifa_ranking-2024-04-04.csv')
rankining_FIFA.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,83.0,Guatemala,GUA,15.0,0.0,83,CONCACAF,1992-12-31
1,32.0,Zambia,ZAM,38.0,0.0,32,CAF,1992-12-31
2,33.0,Portugal,POR,38.0,0.0,33,UEFA,1992-12-31
3,34.0,Austria,AUT,38.0,0.0,34,UEFA,1992-12-31
4,35.0,Colombia,COL,36.0,0.0,35,CONMEBOL,1992-12-31


# Players data_ pre analysis

In [182]:
# Standarizing the names and vowels from dataframes in order to merge them.
# For dataset with the list of players selected to play the championship.
for col in new_df.columns:
    if new_df[col].dtype == object:  # Check column type 'object' (string)
        new_df[col] = new_df[col].str.replace('á', 'a')
        new_df[col] = new_df[col].str.replace('é', 'e')
        new_df[col] = new_df[col].str.replace('í', 'i')
        new_df[col] = new_df[col].str.replace('ó', 'o')
        new_df[col] = new_df[col].str.replace('ú', 'u')
        new_df[col] = new_df[col].str.replace('ï', 'i')
        new_df[col] = new_df[col].str.replace('Á', 'A')

new_df.replace('Brasil','Brazil', inplace=True)
new_df.rename(columns={'Country':'Nation'}, inplace=True)

# For the dataset with the total list of players that are in the FC24 game.
for col in player_rating.columns:
    if player_rating[col].dtype == object:  # Check column type 'object' (string)
        player_rating[col] = player_rating[col].str.replace('á', 'a')
        player_rating[col] = player_rating[col].str.replace('é', 'e')
        player_rating[col] = player_rating[col].str.replace('í', 'i')
        player_rating[col] = player_rating[col].str.replace('ó', 'o')
        player_rating[col] = player_rating[col].str.replace('ú', 'u')
        player_rating[col] = player_rating[col].str.replace('ï', 'i')
        player_rating[col] = player_rating[col].str.replace('Á', 'A')
        new_df.replace('Vinicius Junior','Vini Jr.', inplace=True)

In [183]:
players_ = new_df.merge(player_rating[['Name','Nation','Club', 'Position', 'Age', 'Overall','Preferred foot']], on=('Name','Nation'), how='left')

# Getting rid of the columns that won't be used or that are repeated
players_['Position_y'] = players_['Position_y'].fillna(players_['Position_x'])
players_.rename(columns={'Position_y':'Position'}, inplace=True)
players_.drop(columns=['Team', 'Position_x'], inplace=True)



players_.head()

,Nation,Name,Club,Position,Age,Overall,Preferred foot
0,Argentina,Emiliano Martinez,Aston Villa,GK,31.0,85.0,Right
1,Argentina,Franco Armani,River Plate,GK,36.0,77.0,Right
2,Argentina,Geronimo Rulli,Ajax,GK,31.0,81.0,Right
3,Argentina,Gonzalo Montiel,Nott'm Forest,RB,26.0,79.0,Right
4,Argentina,Nahuel Molina,Atletico de Madrid,RB,25.0,82.0,Right


In [184]:
list_players = new_df.groupby('Nation').count().reset_index()
list_players_by_nation = list_players[['Nation', 'Name']]
print('Old list of players:')
print(list_players_by_nation)
print('-'*50)
players_nation = players_.groupby('Nation').count().reset_index()
players_count_by_nation = players_nation[['Nation', 'Name']]
print('New list of players:')
print(players_count_by_nation)

Old list of players:
            Nation  Name
0        Argentina    29
1          Bolivia    28
2           Brazil    26
3           Canada    27
4            Chile    46
5         Colombia    28
6       Costa Rica    26
7          Ecuador    26
8   Estados Unidos    27
9          Jamaica    26
10          Mexico    31
11          Panama    27
12        Paraguay    27
13            Peru    16
14         Uruguay    20
15       Venezuela    47
--------------------------------------------------
New list of players:
            Nation  Name
0        Argentina    29
1          Bolivia    28
2           Brazil    32
3           Canada    27
4            Chile    46
5         Colombia    28
6       Costa Rica    26
7          Ecuador    26
8   Estados Unidos    27
9          Jamaica    26
10          Mexico    31
11          Panama    27
12        Paraguay    27
13            Peru    16
14         Uruguay    20
15       Venezuela    47


For the Brazil team, the number of players has increased, these could be because of repeated names for the same country that are in the player_rating dataset.

In [185]:
# Checking all the Brazilian players
players_[players_.Nation=='Brazil']

,Nation,Name,Club,Position,Age,Overall,Preferred foot
350,Brazil,Alisson,Liverpool,GK,31.0,89.0,Right
351,Brazil,Bento,NaN,GK,NaN,NaN,NaN
352,Brazil,Rafael,NaN,GK,NaN,NaN,NaN
353,Brazil,Danilo,Juventus,CB,32.0,81.0,Right
354,Brazil,Danilo,Nott'm Forest,CDM,22.0,76.0,Left
355,Brazil,Danilo,VfL Bochum,LB,31.0,76.0,Left
356,Brazil,Danilo,Rangers,ST,24.0,74.0,Right
357,Brazil,Yan Couto,Girona FC,RM,21.0,72.0,Right
358,Brazil,Guilherme Arana,NaN,CB,NaN,NaN,NaN
359,Brazil,Wendell,FC Porto,LB,30.0,76.0,Left


In [186]:
# Drop duplicates in the players_ DataFrame based on the 'Name' column and keeping only the first as they're the ones with highest Overall score.
players_.drop_duplicates(subset=['Name'], keep='first', inplace=True)

players_[players_.Nation=='Brazil']

,Nation,Name,Club,Position,Age,Overall,Preferred foot
350,Brazil,Alisson,Liverpool,GK,31.0,89.0,Right
351,Brazil,Bento,NaN,GK,NaN,NaN,NaN
352,Brazil,Rafael,NaN,GK,NaN,NaN,NaN
353,Brazil,Danilo,Juventus,CB,32.0,81.0,Right
357,Brazil,Yan Couto,Girona FC,RM,21.0,72.0,Right
358,Brazil,Guilherme Arana,NaN,CB,NaN,NaN,NaN
359,Brazil,Wendell,FC Porto,LB,30.0,76.0,Left
360,Brazil,Beraldo,NaN,CB,NaN,NaN,NaN
361,Brazil,Éder Militão,Real Madrid,CB,25.0,86.0,Right
362,Brazil,Gabriel Magalhães,NaN,CB,NaN,NaN,NaN


It can be seen that there are many players with null values. The reason for this can be:


*   Players league is unavailable in the FC24 game, so they are not in the player_rating dataset.
* The player's name is written differently than the one in the player_rating dataset.

I'm going to check this and for the second option change the name to how it is shown in the game

In [187]:
# Counting the numbers of player with null values per each coountry
player_null = players_[players_['Club'].isnull()]
player_null = (player_null.groupby('Nation').count().reset_index())
player_null.rename(columns={'Name':'Count'}, inplace=True)
player_null = player_null[['Nation', 'Count']]
print('Total null values:', player_null['Count'].sum())
player_null

Total null values: 234


,Nation,Count
0,Argentina,1
1,Bolivia,14
2,Brazil,8
3,Canada,7
4,Chile,25
5,Colombia,11
6,Costa Rica,22
7,Ecuador,11
8,Estados Unidos,27
9,Jamaica,15


In [54]:
# Creating a list of the player wtith null values
player_name = players_[players_['Club'].isnull()]
player_name.groupby('Nation')
player_name = player_name[['Nation', 'Name']]
player_name = list(player_name.values)

In [61]:
# Using AI model to browse for the full name of each player
player_response = model.generate_content(f'''
For the list of football players in the following list {player_name}, I need you to adjust their names to their real names and how they are known as. For example, "Rodrigo De Paul" the real name is  "Rodrigo Javier De Paul," and "Gabriel Magalhães" is known as "Gabriel"
Display the full list {player_name}, with just full names and short name, in a dictionary format (players_full_name). Ensure that the adjustments maintain accuracy and consistency, as this is crucial for properdata analysis.''')
to_markdown(player_response.text)

> ```
> players_full_name = {
>     "Rodrigo De Paul": "Rodrigo Javier De Paul",
>     "Anderson Santamaria": "Anderson Santamaría",
>     "Oliver Sonne": "Oliver Sonne",
>     "Pedro Quispe": "Pedro Quispe",
>     "Andre Carrillo": "André Carrillo",
>     "Bryan Reyna": "Bryan Reyna",
>     "Vicente Reyes": "Vicente Reyes",
>     "Mauricio Isla": "Mauricio Andrés Isla Isla",
>     "Benjamin Kuscevic": "Benjamin Kuscevic",
>     "Igor Lichnovsky": "Igor Lichnovsky",
>     "Gary Medel": "Gary Alexander Medel Soto",
>     "Nicolas Diaz": "Nicolás Ignacio Díaz",
>     "Sebastian Vegas": "Sebastián Vegas",
>     "Erick Pulgar": "Erick Antonio Pulgar Farfán",
>     "Ulises Ortegoza": "Ulises Darío Ortegoza",
>     "Williams Alarcon": "Williams Junior Alarcón Rojas",
>     "Jean Meneses": "Jean David Meneses Gutiérrez",
>     "Victor Mendez": "Víctor Felipe Méndez Zepeda",
>     "Luciano Cabral": "Luciano Cabral",
>     "Maximiliano Guerrero": "Maximiliano Guerrero",
>     "Cesar Perez": "César Ariel Pérez Rivas",
>     "Felipe Loyola": "Felipe Loyola",
>     "Alexis Sanchez": "Alexis Alejandro Sánchez",
>     "Eduardo Vargas": "Eduardo Jesús Vargas Rojas",
>     "Diego Valdes": "Diego Antonio Valdés",
>     "Benjamin Brereton": "Benjamín Antonio Brereton Díaz",
>     "Victor Davila": "Víctor Felipe Dávila",
>     "Dario Osorio": "Darío Osorio",
>     "Diego Valencia": "Diego Antonio Valencia",
>     "Steffan Pino": "Steffan Ignacio Pino Pino",
>     "Lucas Assadi": "Lucas Assadi",
>     "Thomas McGill": "Thomas McGill",
>     "Gregoire Swiderski": "Grzegorz Swiderski",
>     "Moise Bombito": "Moïse Bombito",
>     "Luc de Fougerolles": "Luc de Fougerolles",
>     "Kyle Hiebert": "Kyle Hiebert",
>     "Thelonius Bair": "Thelonius Bair",
>     "Junior Hoilett": "Junior Hoilett",
>     "Angel Malagon": "Ángel Malagón",
>     "Jose Raul Rangel": "José Raúl Rangel",
>     "Julio Gonzalez": "Julio González",
>     "Israel Reyes": "Israel Reyes",
>     "Brian Garcia": "Brian García",
>     "Victor Guzman": "Víctor Guzmán Patrón",
>     "Alexis Peña": "Alexis Peña",
>     "Jesus Orozco": "Jesús Corona",
>     "Bryan Gonzalez": "Bryan González",
>     "Luis Romo": "Luis Romo",
>     "Erick Sanchez": "Erick Sánchez",
>     "Roberto Alvarado": "Roberto Alvarado",
>     "Luis Chavez": "Luis Gerardo Chávez",
>     "Angel Montaño": "Angel Montaño",
>     "Fernando Beltran": "Fernando Beltrán",
>     "Carlos Rodriguez": "Carlos Rodríguez",
>     "Marcelo Flores": "Marcelo Flores",
>     "Cesar Huerta": "César Huerta",
>     "Julian Quiñones": "Julián Quiñones",
>     "Alexis Vega": "Alexis Vega",
>     "Uriel Antuna": "Uriel Antuna",
>     "Guillermo Martinez": "Guillermo Martínez",
>     "Diego Lainez": "Diego Lainez",
>     "Felix Torres": "Félix Torres",
>     "Jose Hurtado": "José Hurtado",
>     "William Pacho": "William Pacho",
>     "Andres Micolta": "Andrés Micolta",
>     "Layan Loor": "Layan Loor",
>     "Kendry Paez": "Kendry Páez",
>     "Angel Mena": "Ángel Mena",
>     "Alan Franco": "Alan Franco",
>     "John Yeboah": "John Yeboah",
>     "Enner Valencia": "Enner Valencia",
>     "Jordy Caicedo": "Jordy Caicedo",
>     "Rafael Romo": "Rafael Romo",
>     "Joel Graterol": "Joel Graterol",
>     "Jose David Contreras": "José David Contreras",
>     "Nahuel Ferraresi": "Nahuel Ferraresi",
>     "Carlos Viva": "Carlos José Belloso",
>     "Diego Luna": "Diego Luna",
>     "Teo Quintero": "Teófilo Quintero",
>     "Jhon Chancellor": "Jhon Chancellor",
>     "Miguel Navarro": "Miguel Navarro",
>     "Renne Rivas": "Renne Rivas",
>     "Roberto Rosales": "Roberto Rosales",
>     "Delvin Alfonzo": "Delvin Alfonzo",
>     "Cristian Casseres": "Cristian Casseres",
>     "Bryant Ortega": "Bryant Ortega",
>     "Tomas Rincon": "Tomás Rincón",
>     "Edson Castillo": "Edson Castillo",
>     "Telasco Segovia": "Telasco Segovia",
>     "Eduard Bello": "Eduard Bello",
>     "Kervin Andrade": "Kervin Andrade",
>     "Jhon Murillo": "Jhon Murillo",
>     "Yeferson Soteldo": "Yeferson Soteldo",
>     "Enrique Peña Zuaner": "Enrique Peña Zuaner",
>     "Freddy Vargas": "Freddy Vargas",
>     "Shaquan Davis": "Shaquan Davis",
>     "Jayden Hibbert": "Jayden Hibbert",
>     "Jahmali Waite": "Jahmali Waite",
>     "Richard King": "Richard King",
>     "Tayvon Gray": "Tayvon Gray",
>     "Jon Bell": "Jon Bell",
>     "Bobby Reid": "Bobby Reid",
>     "Alex Marshall": "Alex Marshall",
>     "Adrian Reid": "Adrian Reid",
>     "Kevon Lambert": "Kevon Lambert",
>     "Shamar Nicholson": "Shamar Nicholson",
>     "Renaldo Cephas": "Renaldo Cephas",
>     "Deshane Beckford": "Deshane Beckford",
>     "Kaheim Dixon": "Kaheim Dixon",
>     "Devonte Campbell": "Devonte Campbell",
>     "Ethan Horvath": "Ethan Horvath",
>     "Sean Johnson": "Sean Johnson",
>     "Matt Turner": "Matthew Charles Turner",
>     "Cameron Carter-Vickers": "Cameron Carter-Vickers",
>     "Kristoffer Lund": "Kristoffer Lund",
>     "Mark McKenzie": "Mark McKenzie",
>     "Shaq Moore": "Shaq Moore",
>     "Tim Ream": "Timothy Michael Ream",
>     "Chris Richards": "Chris Richards",
>     "Antonee Robinson": "Antonee Robinson",
>     "Miles Robinson": "Miles Robinson",
>     "Joe Scally": "Joseph Michael Scally",
>     "Tyler Adams": "Tyler Adams",
>     "Johnny Cardoso": "Johnny Cardoso",
>     "Luca de la Torre": "Luca de la Torre",
>     "Weston McKennie": "Weston McKennie",
>     "Yunus Musah": "Yunus Dimoara Musah",
>     "Gio Reyna": "Giovanni Reyna",
>     "Malik Tillman": "Malik Tillman",
>     "Timmy Tillman": "Timmy Tillman",
>     "Brenden Aaronson": "Brenden Aaronson",
>     "Folarin Balogun": "Folarin Balogun",
>     "Ricardo Pepi": "Ricardo Pepi",
>     "Christian Pulisic": "Christian Mate Pulisic",
>     "Josh Sargent": "Joshua Thomas Sargent",
>     "Tim Weah": "Timothy Tarpeh Weah",
>     "Haji Wright": "Haji Wright",
>     "Sergio Rochet": "Sergio Rochet",
>     "Santiago Mele": "Santiago Mele",
>     "Sebastian Caceres": "Sebastián Cáceres",
>     "Lucas Olaza": "Lucas Olaza",
>     "Maximiliano Araujo": "Maximiliano Araújo",
>     "Brian Rodriguez": "Brian Rodriguez",
>     "Orlando Mosquera": "Orlando Mosquera",
>     "Luis Mejia": "Luis Mejía",
>     "Cesar Samudio": "Cesar Samudio",
>     "Fidel Escobar": "Fidel Escobar",
>     "Eduardo Anderson": "Eduardo Anderson",
>     "Jose Cordoba": "José Córdoba",
>     "Eric Davis": "Eric Davis",
>     "Cesar Blackman": "César Blackman",
>     "Edgardo Fariña": "Edgardo Fariña",
>     "Roderick Miller": "Roderick Miller",
>     "Martin Krug": "Martín Krug",
>     "Cristian Martinez": "Cristian Martinez",
>     "Edgar Barcenas": "Edgar Barcenas",
>     "Jovani Welch": "Jovani Welch",
>     "Freddy Gondola": "Freddy Gondola",
>     "Carlos Harvey": "Carlos Harvey",
>     "Abidel Ararza": "Abidel Araráz",
>     "Cesar Yanis": "César Yanis",
>     "Ismael Diaz": "Ismael Díaz",
>     "Jose Fajardo": "José Fajardo",
>     "Eduardo Guerrero": "Eduardo Guerrero",
>     "Cecilio Waterman": "Cecilio Waterman",
>     "Gustavo Almada": "Gustavo Almada",
>     "Diego Medina": "Diego Medina",


In [197]:
import ast
# Access the content attribute from the response
player_content = player_response.candidates[0].content.parts[0].text

# Find the starting index of 'player_dict ='
start_index = player_content.find('players_full_name = ')

# Find the ending index of ']\n```'
end_index = player_content.find('```', start_index)

# Extract the substring containing player_list
player_list_str = player_content[start_index:end_index].strip()+ '}'
# Evaluate the string as Python code to get the player_list dictionary
exec(player_list_str)

# Flatten the nested dictionary and convert to a DataFrame
players_data = [(full_name, name) for full_name, name in players_full_name.items()]

df_player_fullname = pd.DataFrame(players_data, columns=['short Name', 'Full Name'])
df_player_fullname.dropna

# Giving same standar to the list of names
for col in df_player_fullname.columns:
    if df_player_fullname[col].dtype == object:  # Check column type 'object' (string)
        df_player_fullname[col] = df_player_fullname[col].str.replace('á', 'a')
        df_player_fullname[col] = df_player_fullname[col].str.replace('é', 'e')
        df_player_fullname[col] = df_player_fullname[col].str.replace('í', 'i')
        df_player_fullname[col] = df_player_fullname[col].str.replace('ó', 'o')
        df_player_fullname[col] = df_player_fullname[col].str.replace('ú', 'u')
        df_player_fullname[col] = df_player_fullname[col].str.replace('ï', 'i')
        df_player_fullname[col] = df_player_fullname[col].str.replace('Á', 'A')

# Display the DataFrame
print(df_player_fullname)

              short Name               Full Name
0        Rodrigo De Paul  Rodrigo Javier De Paul
1    Anderson Santamaria     Anderson Santamaria
2           Oliver Sonne            Oliver Sonne
3           Pedro Quispe            Pedro Quispe
4         Andre Carrillo          Andre Carrillo
..                   ...                     ...
162         Jose Fajardo            Jose Fajardo
163     Eduardo Guerrero        Eduardo Guerrero
164     Cecilio Waterman        Cecilio Waterman
165       Gustavo Almada          Gustavo Almada
166         Diego Medina            Diego Medina

[167 rows x 2 columns]


In [198]:
# Merging tables
players_ = players_.merge(df_player_fullname[['short Name', 'Full Name']], on=('short Name'), how='left')
players_[players_['Full Name'].notnull()]

,Nation,short Name,Club,Position,Age,Overall,Preferred foot,Full Name
17,Argentina,Rodrigo De Paul,NaN,CM,NaN,NaN,NaN,Rodrigo Javier De Paul
34,Peru,Anderson Santamaria,NaN,CB,NaN,NaN,NaN,Anderson Santamaria
35,Peru,Oliver Sonne,NaN,CB,NaN,NaN,NaN,Oliver Sonne
40,Peru,Pedro Quispe,NaN,CM,NaN,NaN,NaN,Pedro Quispe
42,Peru,Andre Carrillo,NaN,CF,NaN,NaN,NaN,Andre Carrillo
...,...,...,...,...,...,...,...,...
314,Panama,Jose Fajardo,NaN,CF,NaN,NaN,NaN,Jose Fajardo
315,Panama,Eduardo Guerrero,NaN,CF,NaN,NaN,NaN,Eduardo Guerrero
316,Panama,Cecilio Waterman,NaN,CF,NaN,NaN,NaN,Cecilio Waterman
319,Bolivia,Gustavo Almada,NaN,GK,NaN,NaN,NaN,Gustavo Almada


In [202]:
# Rename column Name in Fifa dataset to be able to match
player_rating.rename(columns={'Name':'Full Name'}, inplace=True)

# Recheck values with database from FC 24 game
df_merged = players_.merge(player_rating[['Full Name', 'Club',	'Position',	'Age',	'Overall',	'Preferred foot']], on=('Full Name'), how='left')

# turn back as it was
player_rating.rename(columns={'Full Name':'Name'}, inplace=True)

# Fill NaN values from player_rating into the merged DataFrame
df_merged['Club'] = df_merged['Club_x'].fillna(df_merged['Club_y'])
df_merged['Position'] = df_merged['Position_x'].fillna(df_merged['Position_y'])
df_merged['Age'] = df_merged['Age_x'].fillna(df_merged['Age_y'])
df_merged['Overall'] = df_merged['Overall_x'].fillna(df_merged['Overall_y'])
df_merged['Preferred foot'] = df_merged['Preferred foot_x'].fillna(df_merged['Preferred foot_y'])
df_merged['Full Name'] = df_merged['Full Name'].fillna(df_merged['short Name'])

# Drop redundant columns
df_merged = df_merged.drop(columns=['Club_y', 'Position_y', 'Age_y', 'Overall_y', 'Preferred foot_y'])
df_merged = df_merged.drop(columns=['Club_x', 'Position_x', 'Age_x', 'Overall_x', 'Preferred foot_x'])

df_merged[df_merged.Nation == 'Peru']

,Nation,short Name,Full Name,Club,Position,Age,Overall,Preferred foot
29,Peru,Pedro Gallese,Pedro Gallese,Orlando City,GK,33.0,73.0,Right
30,Peru,Luis Advincula,Luis Advincula,Boca Juniors,RB,33.0,75.0,Right
31,Peru,Miguel Araujo,Miguel Araujo,Portland Timbers,CB,28.0,67.0,Right
32,Peru,Alexander Callens,Alexander Callens,AEK Athens,CB,31.0,72.0,Left
33,Peru,Marcos Lopez,Marcos Lopez,Feyenoord,LB,23.0,70.0,Left
34,Peru,Anderson Santamaria,Anderson Santamaria,NaN,CB,NaN,NaN,NaN
35,Peru,Oliver Sonne,Oliver Sonne,Silkeborg IF,CB,22.0,68.0,Right
36,Peru,Luis Abram,Luis Abram,Atlanta United,CB,27.0,72.0,Left
37,Peru,Wilder Cartagena,Wilder Cartagena,Orlando City,CDM,29.0,69.0,Right
38,Peru,Jesus Castillo,Jesus Castillo,Gil Vicente,CDM,22.0,70.0,Right


In [204]:
# Counting the numbers of player with null values per each coountry
player_null = df_merged[df_merged['Overall'].isnull()]
player_null = (player_null.groupby('Nation').count().reset_index())
player_null.rename(columns={'short Name':'Count'}, inplace=True)
player_null = player_null[['Nation', 'Count']]
print('Total null values:', player_null['Count'].sum())
player_null

Total null values: 210


,Nation,Count
0,Bolivia,14
1,Brazil,8
2,Canada,7
3,Chile,25
4,Colombia,11
5,Costa Rica,22
6,Ecuador,10
7,Estados Unidos,11
8,Jamaica,13
9,Mexico,21


It is possible to see that the number of null values continues to be high

## Fifa 23 dataset

I am going to use the data of the same game but from last year, which has more complete info.

In [ ]:
! kaggle datasets download -d sanjeetsinghnaik/fifa-23-players-dataset

In [ ]:
! unzip fifa-23-players-dataset.zip

In [94]:
# Checking the info contained in this new dataset
fifa_23 = pd.read_csv('Fifa 23 Players Data.csv')
fifa_23.head()


,Known As,Full Name,Overall,Potential,Value(in Euro),Positions Played,Best Position,Nationality,Image Link,Age,...,LM Rating,CM Rating,RM Rating,LWB Rating,CDM Rating,RWB Rating,LB Rating,CB Rating,RB Rating,GK Rating
0,L. Messi,Lionel Messi,91,91,54000000,RW,CAM,Argentina,https://cdn.sofifa.net/players/158/023/23_60.png,35,...,91,88,91,67,66,67,62,53,62,22
1,K. Benzema,Karim Benzema,91,91,64000000,"CF,ST",CF,France,https://cdn.sofifa.net/players/165/153/23_60.png,34,...,89,84,89,67,67,67,63,58,63,21
2,R. Lewandowski,Robert Lewandowski,91,91,84000000,ST,ST,Poland,https://cdn.sofifa.net/players/188/545/23_60.png,33,...,86,83,86,67,69,67,64,63,64,22
3,K. De Bruyne,Kevin De Bruyne,91,91,107500000,"CM,CAM",CM,Belgium,https://cdn.sofifa.net/players/192/985/23_60.png,31,...,91,91,91,82,82,82,78,72,78,24
4,K. Mbappé,Kylian Mbappé,91,95,190500000,"ST,LW",ST,France,https://cdn.sofifa.net/players/231/747/23_60.png,23,...,92,84,92,70,66,70,66,57,66,21


In [95]:
for col in fifa_23.columns:
    if fifa_23[col].dtype == object:  # Check column type 'object' (string)
        fifa_23[col] = fifa_23[col].str.replace('á', 'a')
        fifa_23[col] = fifa_23[col].str.replace('é', 'e')
        fifa_23[col] = fifa_23[col].str.replace('í', 'i')
        fifa_23[col] = fifa_23[col].str.replace('ó', 'o')
        fifa_23[col] = fifa_23[col].str.replace('ú', 'u')
        fifa_23[col] = fifa_23[col].str.replace('ï', 'i')
        fifa_23[col] = fifa_23[col].str.replace('Á', 'A')

In [209]:
# Changing columns name
fifa_23.rename(columns={'Name':'Full Name'}, inplace=True)

In [211]:
# Mergin las dataset with information from FIFA 2023
df = df_merged.merge(fifa_23[['Full Name', 'Overall', 'Value(in Euro)', 'Best Position',  'Age', 'Height(in cm)', 'Weight(in kg)', 'Club Name', 'Preferred Foot']], on=('Full Name'), how='left')

# Fill NaN values
df['Club'] = df['Club'].fillna(df['Club Name'])
df['Position'] = df['Position'].fillna(df['Best Position'])
df['Age_x'] = df['Age_x'].fillna(df['Age_y'])
df['Overall_x'] = df['Overall_x'].fillna(df['Overall_y'])
df['Preferred foot'] = df['Preferred foot'].fillna(df['Preferred Foot'])

# Drop redundant columns
df = df.drop(columns=['Overall_y', 'Best Position', 'Age_y', 'Club Name', 'Preferred Foot'])

# Rename columns
df = df.rename(columns={
    'Age_x': 'Age',
    'Overall_x': 'Overall'})

# Drop duplicated values
df.drop_duplicates(subset=['Full Name'], keep='first', inplace=True)

df[df.Nation == 'Brazil']

,Nation,short Name,Full Name,Club,Position,Age,Overall,Preferred foot,Value(in Euro),Height(in cm),Weight(in kg)
355,Brazil,Alisson,Alisson,Liverpool,GK,31.0,89.0,Right,NaN,NaN,NaN
356,Brazil,Bento,Bento,NaN,GK,NaN,NaN,NaN,NaN,NaN,NaN
357,Brazil,Rafael,Rafael,NaN,GK,NaN,NaN,NaN,NaN,NaN,NaN
358,Brazil,Danilo,Danilo,Juventus,CB,32.0,81.0,Right,NaN,NaN,NaN
359,Brazil,Yan Couto,Yan Couto,Girona FC,RM,21.0,72.0,Right,NaN,NaN,NaN
360,Brazil,Guilherme Arana,Guilherme Arana,NaN,CB,NaN,NaN,NaN,NaN,NaN,NaN
361,Brazil,Wendell,Wendell,FC Porto,LB,30.0,76.0,Left,NaN,NaN,NaN
362,Brazil,Beraldo,Beraldo,NaN,CB,NaN,NaN,NaN,NaN,NaN,NaN
363,Brazil,Éder Militão,Éder Militão,Real Madrid,CB,25.0,86.0,Right,NaN,NaN,NaN
364,Brazil,Gabriel Magalhães,Gabriel Magalhães,NaN,CB,NaN,NaN,NaN,NaN,NaN,NaN


For all the Brazilian players, it is possible to appreciate that they are not found in FIFA 23 database by their full name. I will try by using the column 'know as'.

In [213]:
# Counting the numbers of player with null values per each coountry
player_null_ = df[df['Overall'].isnull()]
player_null_ = (player_null_.groupby('Nation').count().reset_index())
player_null_.rename(columns={'Full Name':'Count'}, inplace=True)
player_null_ = player_null_[['Nation', 'Count']]
print('Total null values:', player_null_['Count'].sum())
player_null_

Total null values: 170


,Nation,Count
0,Bolivia,10
1,Brazil,8
2,Canada,6
3,Chile,25
4,Colombia,9
5,Costa Rica,20
6,Ecuador,6
7,Estados Unidos,11
8,Jamaica,13
9,Mexico,14


The number of null values has decreased significantly

In [215]:
fifa_23.rename(columns={'Known As':'short Name'}, inplace=True)

In [216]:
# Mergin last with short names  from FIFA 2023
df = df.merge(fifa_23[['short Name', 'Overall', 'Value(in Euro)', 'Best Position',  'Age', 'Height(in cm)', 'Weight(in kg)', 'Club Name', 'Preferred Foot']], on=('short Name'), how='left')

# Fill NaN values from player_rating into the merged DataFrame
df['Club'] = df['Club'].fillna(df['Club Name'])
df['Position'] = df['Position'].fillna(df['Best Position'])
df['Age_x'] = df['Age_x'].fillna(df['Age_y'])
df['Overall_x'] = df['Overall_x'].fillna(df['Overall_y'])
df['Preferred foot'] = df['Preferred foot'].fillna(df['Preferred Foot'])
df['Value(in Euro)_x'] = df['Value(in Euro)_x'].fillna(df['Value(in Euro)_y'])
df['Height(in cm)_x'] = df['Height(in cm)_x'].fillna(df['Height(in cm)_y'])
df['Weight(in kg)_x'] = df['Weight(in kg)_x'].fillna(df['Weight(in kg)_y'])

# Drop redundant columns
df = df.drop(columns=['Overall_y', 'Best Position', 'Age_y', 'Club Name', 'Preferred Foot','Value(in Euro)_y','Height(in cm)_y','Weight(in kg)_y'])

# Rename columns
df = df.rename(columns={
    'Age_x': 'Age',
    'Overall_x': 'Overall',
    'Height(in cm)_x': 'Height(in cm)',
    'Weight(in kg)_x': 'Weight(in kg)',
    'Value(in Euro)_x':'Value(in Euro)'})

# Drop any duplicated value
df.drop_duplicates(subset=['short Name'], keep='first', inplace=True)

df[df.Nation == 'Brazil']

,Nation,short Name,Full Name,Club,Position,Age,Overall,Preferred foot,Value(in Euro),Height(in cm),Weight(in kg)
345,Brazil,Alisson,Alisson,Liverpool,GK,31.0,89.0,Right,79000000.0,191.0,91.0
346,Brazil,Bento,Bento,NaN,GK,NaN,NaN,NaN,NaN,NaN,NaN
347,Brazil,Rafael,Rafael,NaN,GK,NaN,NaN,NaN,NaN,NaN,NaN
348,Brazil,Danilo,Danilo,Juventus,CB,32.0,81.0,Right,18000000.0,184.0,78.0
351,Brazil,Yan Couto,Yan Couto,Girona FC,RM,21.0,72.0,Right,4099999.0,168.0,60.0
352,Brazil,Guilherme Arana,Guilherme Arana,NaN,CB,NaN,NaN,NaN,NaN,NaN,NaN
353,Brazil,Wendell,Wendell,FC Porto,LB,30.0,76.0,Left,5000000.0,176.0,70.0
354,Brazil,Beraldo,Beraldo,NaN,CB,NaN,NaN,NaN,NaN,NaN,NaN
355,Brazil,Éder Militão,Éder Militão,Real Madrid,CB,25.0,86.0,Right,57500000.0,186.0,78.0
356,Brazil,Gabriel Magalhães,Gabriel Magalhães,NaN,CB,NaN,NaN,NaN,NaN,NaN,NaN


In [217]:
# Counting the numbers of player with null values per each coountry
player_null_ = df[df['Overall'].isnull()]
player_null_ = (player_null_.groupby('Nation').count().reset_index())
player_null_.rename(columns={'short Name':'Count'}, inplace=True)
player_null_ = player_null_[['Nation', 'Count']]
print('Total null values:', player_null_['Count'].sum())
player_null_

Total null values: 167


,Nation,Count
0,Bolivia,9
1,Brazil,8
2,Canada,6
3,Chile,24
4,Colombia,9
5,Costa Rica,20
6,Ecuador,6
7,Estados Unidos,11
8,Jamaica,13
9,Mexico,14


## Filling null values (iterative_imputer) - Not run ask Gaby

In [159]:
# Select only numeric columns for imputation
numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns

# Initialize IterativeImputer
imputer = IterativeImputer(max_iter=10, random_state=0)

# Fit and transform the data
df[numeric_columns] = imputer.fit_transform(df[numeric_columns])

# Columns with null values
df.isna().sum()

Nation              0
short Name          0
Full Name           0
Club              153
Position            0
Age                 0
Overall             0
Preferred foot    153
Value(in Euro)      0
Height(in cm)       0
Weight(in kg)       0
dtype: int64

In [173]:
# Impute missing values in Preferred foot column
most_preferred_foot = df['Preferred foot'].mode()[0]
df['Preferred foot'] = df['Preferred foot'].fillna(most_preferred_foot)

# Print the DataFrame after imputing categorical values
df.isna().sum()

Nation              0
short Name          0
Full Name           0
Club              153
Position            0
Age                 0
Overall             0
Preferred foot      0
Value(in Euro)      0
Height(in cm)       0
Weight(in kg)       0
dtype: int64

In [174]:
players_nation_mean = df.groupby('Nation').mean('Overall').reset_index()
players_nation_mean

,Nation,Age,Overall,Value(in Euro),Height(in cm),Weight(in kg)
0,Argentina,27.310345,80.172414,2.504169e+07,179.735828,74.444647
1,Bolivia,26.181319,68.051123,5.773141e+06,180.863873,74.972261
2,Brazil,25.917160,78.578853,2.462038e+07,180.158666,74.651368
3,Canada,25.977208,71.603245,8.197295e+06,180.382735,75.544487
4,Chile,27.232441,71.002181,5.146586e+06,179.703107,74.489915
5,Colombia,26.967033,74.551123,1.091546e+07,180.097855,75.583427
6,Costa Rica,26.331361,71.331747,7.732228e+06,180.216113,74.442737
7,Ecuador,25.215385,71.291505,6.197198e+06,178.677426,73.290612
8,Estados Unidos,24.672365,73.959495,8.427518e+06,183.182148,77.589288
9,Jamaica,26.115385,69.267559,6.718190e+06,180.714058,74.855572


In [176]:
df[df.Nation == 'Panama']

,Nation,short Name,Full Name,Club,Position,Age,Overall,Preferred foot,Value(in Euro),Height(in cm),Weight(in kg)
290,Panama,Orlando Mosquera,Orlando Mosquera,NaN,GK,26.230769,72.381271,Right,9.302996e+06,180.271433,74.908507
291,Panama,Luis Mejia,Luis Mejia,Union Española,GK,31.000000,73.000000,Left,1.600000e+06,193.000000,81.000000
292,Panama,Cesar Samudio,Cesar Samudio,NaN,GK,26.230769,72.381271,Right,9.302996e+06,180.271433,74.908507
293,Panama,Fidel Escobar,Fidel Escobar,NaN,CB,26.230769,72.381271,Right,9.302996e+06,180.271433,74.908507
294,Panama,Eduardo Anderson,Eduardo Anderson,NaN,CB,26.230769,72.381271,Right,9.302996e+06,180.271433,74.908507
295,Panama,Jose Cordoba,Jose Cordoba,NaN,CB,26.230769,72.381271,Right,9.302996e+06,180.271433,74.908507
296,Panama,Eric Davis,Eric Davis,NaN,CB,26.230769,72.381271,Right,9.302996e+06,180.271433,74.908507
297,Panama,Ivan Anderson,Ivan Anderson,Monagas SC,RB,25.000000,66.000000,Right,9.302996e+06,180.292871,74.773386
298,Panama,Michael Murillo,Michael Murillo,OM,RB,27.000000,72.000000,Right,3.500000e+06,183.000000,75.000000
299,Panama,Cesar Blackman,Cesar Blackman,NaN,CB,26.230769,72.381271,Right,9.302996e+06,180.271433,74.908507


# Data Exploration Analysis (DEA)

The data frames we have so far are the following:

* df = list of players from web scraping with the values obtained from the FIFA 23 and 24 game database
* hs_df = historical data from past editions of the Copa America
* rankining_FIFA = Ranking FIFA of the different teams participating in the competition

In [238]:
hs_df['Away Team'].unique()

array(['Uruguay', 'Chile', 'Brazil', 'Argentina', 'Paraguay', 'Bolivia',
       'Peru', 'Ecuador', 'Colombia', 'Venezuela', 'Mexico',
       'United States', 'Costa Rica', 'Japan', 'Honduras', 'Jamaica',
       'Haiti', 'Panama', 'Qatar'], dtype=object)

In [241]:
df.Nation.unique()
df.replace('Estados Unidos','United States',inplace=True)

In [242]:
for country in df.Nation.unique():
  if country not in hs_df['Away Team'].unique():
    print(country)


Canada


For Canada, it will be the first time playing Copa America tournament.

In [271]:
# Drop the columns that are not useful
hs_df.drop(columns=['Tournament'] , inplace=True)

# I will only consider the tournaments that were held from 1991 till the last (2021)
hs_df = hs_df[hs_df['Year'] >= 1991]


hs_df.head()

,Year,Date,Home Team,Away Team,Home Score,Away Score,Shootout,City,Country,Neutral Venue,Winning Team,first_shooter,Losing Team
495,1991,1991-07-06,Chile,Venezuela,2.0,0.0,False,Santiago,Chile,False,Chile,NaN,Venezuela
496,1991,1991-07-06,Paraguay,Peru,1.0,0.0,False,Santiago,Chile,True,Paraguay,NaN,Peru
497,1991,1991-07-07,Bolivia,Uruguay,1.0,1.0,False,Valparaíso,Chile,True,Draw,NaN,Draw
498,1991,1991-07-07,Colombia,Ecuador,1.0,0.0,False,Valparaíso,Chile,True,Colombia,NaN,Ecuador
499,1991,1991-07-08,Argentina,Venezuela,3.0,0.0,False,Santiago,Chile,True,Argentina,NaN,Venezuela


In [ ]:
# Measuring the average overall score for each team based on the players listed to play this year tournament
team_avg_overall = df.groupby('Nation').mean('Overall').reset_index()
team_avg_overall

,Nation,Age,Overall,Value(in Euro),Height(in cm),Weight(in kg)
0,Argentina,27.310345,80.172414,2.741852e+07,179.666667,74.333333
1,Bolivia,26.157895,66.000000,1.066667e+06,181.666667,75.000000
2,Brazil,25.777778,81.333333,3.419375e+07,180.062500,74.625000
3,Canada,25.904762,71.380952,7.644444e+06,180.444444,75.833333
4,Chile,28.000000,71.181818,3.052273e+06,180.363636,75.272727
5,Colombia,27.315789,75.578947,1.195882e+07,180.000000,75.941176
6,Costa Rica,26.666667,67.833333,1.135000e+06,180.000000,72.400000
7,Ecuador,24.894737,70.947368,4.126667e+06,177.666667,71.933333
8,Jamaica,26.000000,66.153846,2.582500e+06,181.400000,74.900000
9,Mexico,24.941176,73.058824,6.394118e+06,176.235294,68.882353


In [307]:
# Adding the average team weight in each game played in the past.

played_games = hs_df.merge(team_avg_overall[['Nation', 'Overall']],
                           left_on='Home Team',
                           right_on='Nation',
                           suffixes=('', '_Overall_HT'))

# Drop the redundant 'Nation' column
played_games.drop(columns=['Nation'], inplace=True)

# Second merge: Away Team overall
played_games = played_games.merge(team_avg_overall[['Nation', 'Overall']],
                                  left_on='Away Team',
                                  right_on='Nation',
                                  suffixes=('_HT', '_AT'))

# Drop the redundant 'Nation' column from the second merge
played_games.drop(columns=['Nation'], inplace=True)

# Difference between each team based on the overall
played_games['Overall_Diff'] = played_games['Overall_HT'] - played_games['Overall_AT']

played_games

,Year,Date,Home Team,Away Team,Home Score,Away Score,Shootout,City,Country,Neutral Venue,Winning Team,first_shooter,Losing Team,Overall_HT,Overall_AT,Overall_Diff
0,1991,1991-07-06,Chile,Venezuela,2.0,0.0,False,Santiago,Chile,False,Chile,NaN,Venezuela,71.181818,68.818182,2.363636
1,1999,1999-07-03,Chile,Venezuela,3.0,0.0,False,Ciudad del Este,Paraguay,True,Chile,NaN,Venezuela,71.181818,68.818182,2.363636
2,2001,2001-07-14,Chile,Venezuela,1.0,0.0,False,Barranquilla,Colombia,True,Chile,NaN,Venezuela,71.181818,68.818182,2.363636
3,2011,2011-07-17,Chile,Venezuela,1.0,2.0,False,San Juan,Argentina,True,Venezuela,NaN,Chile,71.181818,68.818182,2.363636
4,1991,1991-07-10,Paraguay,Venezuela,5.0,0.0,False,Santiago,Chile,True,Paraguay,NaN,Venezuela,70.684211,68.818182,1.866029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,2007,2007-07-05,Colombia,United States,1.0,0.0,False,Barquisimeto,Venezuela,True,Colombia,NaN,United States,75.578947,74.437500,1.141447
324,1995,1995-07-14,Argentina,United States,0.0,3.0,False,Paysandú,Uruguay,True,United States,NaN,Argentina,80.172414,74.437500,5.734914
325,2007,2007-06-28,Argentina,United States,4.0,1.0,False,Maracaibo,Venezuela,True,Argentina,NaN,United States,80.172414,74.437500,5.734914
326,1993,1993-06-19,Ecuador,United States,2.0,0.0,False,Quito,Ecuador,False,Ecuador,NaN,United States,70.947368,74.437500,-3.490132


In [308]:
# # Changing rank date type to date time
rankining_FIFA['rank_date']=pd.to_datetime(rankining_FIFA['rank_date'])

# # Adding a new column Year
rankining_FIFA['Year']=rankining_FIFA['rank_date'].dt.year

# Replacing Country names to mach with the ones from played_games
rankining_FIFA.replace('USA','United States',inplace=True)

rankining_FIFA.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date,Year
0,83.0,Guatemala,GUA,15.0,0.0,83,CONCACAF,1992-12-31,1992
1,32.0,Zambia,ZAM,38.0,0.0,32,CAF,1992-12-31,1992
2,33.0,Portugal,POR,38.0,0.0,33,UEFA,1992-12-31,1992
3,34.0,Austria,AUT,38.0,0.0,34,UEFA,1992-12-31,1992
4,35.0,Colombia,COL,36.0,0.0,35,CONMEBOL,1992-12-31,1992


In [309]:
# Adding ranking FIFA to played_games dataset
played_games = played_games.merge(rankining_FIFA[['Year', 'rank','country_full']],
                                  left_on=['Year', 'Home Team'],
                                  right_on=['Year', 'country_full'],
                                  suffixes=('', '_rank_FIFA_HT'))
# Drop not necessary columns
played_games.drop(columns=['country_full'] , inplace=True)

# Adding ranking FIFA to played_games dataset
played_games = played_games.merge(rankining_FIFA[['Year', 'rank','country_full']],
                                  left_on=['Year', 'Away Team'],
                                  right_on=['Year', 'country_full'],
                                  suffixes=('_HT', '_AT'))
# Drop not necessary columns
played_games.drop(columns=['country_full'] , inplace=True)


# Difference between each team based on the overall
played_games['rank_FIFA_Diff'] = played_games['_rank_FIFA_HT'] - played_games['_rank_FIFA_AT']

played_games

KeyError: 'rank'

In [310]:
played_games

,Year,Date,Home Team,Away Team,Home Score,Away Score,Shootout,City,Country,Neutral Venue,...,rank_change_x,confederation_x,rank_date_x,rank_y,country_abrv_y,total_points_y,previous_points_y,rank_change_y,confederation_y,rank_date_y
0,1991,1991-07-06,Chile,Venezuela,2.0,0.0,False,Santiago,Chile,False,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,1999,1999-07-03,Chile,Venezuela,3.0,0.0,False,Ciudad del Este,Paraguay,True,...,4.0,CONMEBOL,1999-01-27,126.0,VEN,257.0,19.0,-3.0,CONMEBOL,1999-01-27
2,1999,1999-07-03,Chile,Venezuela,3.0,0.0,False,Ciudad del Este,Paraguay,True,...,4.0,CONMEBOL,1999-01-27,126.0,VEN,257.0,257.0,0.0,CONMEBOL,1999-02-24
3,1999,1999-07-03,Chile,Venezuela,3.0,0.0,False,Ciudad del Este,Paraguay,True,...,4.0,CONMEBOL,1999-01-27,126.0,VEN,255.0,257.0,0.0,CONMEBOL,1999-03-24
4,1999,1999-07-03,Chile,Venezuela,3.0,0.0,False,Ciudad del Este,Paraguay,True,...,4.0,CONMEBOL,1999-01-27,119.0,VEN,277.0,255.0,-7.0,CONMEBOL,1999-04-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34287,1993,1993-06-16,Uruguay,United States,1.0,0.0,False,Ambato,Ecuador,True,...,-1.0,CONMEBOL,1993-12-23,28.0,United States,44.0,43.0,4.0,CONCACAF,1993-08-08
34288,1993,1993-06-16,Uruguay,United States,1.0,0.0,False,Ambato,Ecuador,True,...,-1.0,CONMEBOL,1993-12-23,26.0,United States,45.0,44.0,-2.0,CONCACAF,1993-09-23
34289,1993,1993-06-16,Uruguay,United States,1.0,0.0,False,Ambato,Ecuador,True,...,-1.0,CONMEBOL,1993-12-23,26.0,United States,45.0,45.0,0.0,CONCACAF,1993-10-22
34290,1993,1993-06-16,Uruguay,United States,1.0,0.0,False,Ambato,Ecuador,True,...,-1.0,CONMEBOL,1993-12-23,27.0,United States,45.0,45.0,1.0,CONCACAF,1993-11-19
